In [2]:
import numpy as np

#Membaca data text

with open('data/reviews.txt', 'r') as f:
    reviews = f.read()
with open('data/labels.txt', 'r') as f:
    labels = f.read()

print(reviews[:1000])
print()
print(labels[:20])
    
    

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the whole situation  all remind me of the schools i knew and their students . when i saw the episode in which a student repeatedly tried to burn down the school  i immediately recalled . . . . . . . . . at . . . . . . . . . . high . a classic line inspector i  m here to sack one of your teachers . student welcome to bromwell high . i expect that many adults of my age think that bromwell high is far fetched . what a pity that it isn  t   
story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turn

In [5]:
from string import punctuation

reviews = reviews.lower() #Standarisasi huruf kecil
allText = ''.join([
    c for c in reviews
    if c not in punctuation
])

#Memisahkan dengan baris baru dan spasi

reviewsSplit = allText.split('\n')
allText = ' '.join(reviewsSplit)

#Buat list kosa kata
words = allText.split()

In [6]:
words[:30]

['bromwell',
 'high',
 'is',
 'a',
 'cartoon',
 'comedy',
 'it',
 'ran',
 'at',
 'the',
 'same',
 'time',
 'as',
 'some',
 'other',
 'programs',
 'about',
 'school',
 'life',
 'such',
 'as',
 'teachers',
 'my',
 'years',
 'in',
 'the',
 'teaching',
 'profession',
 'lead',
 'me']

In [7]:
#Encoding kata

from collections import Counter

#Membuat kamus untuk memetakan kata ke integer

counts = Counter(words)
vocab = sorted(counts, key = counts.get, reverse = True)
vocabToInt = {
    word: ii for ii, word in enumerate(vocab, 1)
}

# Gunakan kamus untuk menoken setiap review di dalam reviewsSplit
# Simpan tokneized reviews di reviewsInt

reviewsInts = []
for review in reviewsSplit:
    reviewsInts.append([
        vocabToInt[word]
        for word in review.split()
    ])

In [8]:
#Testing code

#status kamus
print('Kata Unik :', len((vocabToInt)))
print()

#print token di review pertama
print('Review Token : \n', reviewsInts[:1])

Kata Unik : 74072

Review Token : 
 [[21025, 308, 6, 3, 1050, 207, 8, 2138, 32, 1, 171, 57, 15, 49, 81, 5785, 44, 382, 110, 140, 15, 5194, 60, 154, 9, 1, 4975, 5852, 475, 71, 5, 260, 12, 21025, 308, 13, 1978, 6, 74, 2395, 5, 613, 73, 6, 5194, 1, 24103, 5, 1983, 10166, 1, 5786, 1499, 36, 51, 66, 204, 145, 67, 1199, 5194, 19869, 1, 37442, 4, 1, 221, 883, 31, 2988, 71, 4, 1, 5787, 10, 686, 2, 67, 1499, 54, 10, 216, 1, 383, 9, 62, 3, 1406, 3686, 783, 5, 3483, 180, 1, 382, 10, 1212, 13583, 32, 308, 3, 349, 341, 2913, 10, 143, 127, 5, 7690, 30, 4, 129, 5194, 1406, 2326, 5, 21025, 308, 10, 528, 12, 109, 1448, 4, 60, 543, 102, 12, 21025, 308, 6, 227, 4146, 48, 3, 2211, 12, 8, 215, 23]]


In [9]:
#Encoding Labels
#labels bisa Positif atau negatif tergantung 0 atau 1

#  1 = positif, 0 = negatif
labelsSplit = labels.split('\n')
encodedLabels = np.array(
[
    1 if 
    label == 'positive'
    else 0 
    for label in labelsSplit
])

In [11]:
#menghapus Outline

reviewLens = Counter([
    len(x) for x in reviewsInts
])

print("review yang kosong : {}".format(reviewLens[0]))
print("review terpanjang : {}".format(max(reviewLens)))

review yang kosong : 1
review terpanjang : 2514


In [15]:
print('Jumlah Reviews sebelum menghapus outline :', len(reviewsInts))

# Hapus review / label yang kosong dari array reviewsInts 

# get index di review dengan panjang 0

nonZeroIdx = [
    ii for ii,
    review in enumerate(reviewsInts)
    if len(review) != 0
]

#Hapus reviews yang panjang = 0 dan label nya
reviewsInts = [
    reviewsInts[ii]
    for ii in nonZeroIdx
]

encodedLabels = np.array(
[
    encodedLabels[ii] for ii in nonZeroIdx
])

print('Jumlah reviews setelah menghapus outline :',
     len(reviewsInts))

Jumlah Reviews sebelum menghapus outline : 25001
Jumlah reviews setelah menghapus outline : 25000


In [19]:
# buat fungsi untuk padding sequence
def padfeatures(reviewsInts, seqLength):
    
    #menagmbil bentuk baris dan colom yg benar
    features = np.zeros((len(reviewsInts), seqLength), dtype = int)
    
    # setiap reviews ambil review nya
    for i, row in enumerate(reviewsInts):
        features[i, -len(row):] = np.array(row)[:seqLength]
        
    return features

In [20]:
seqLength = 200
features = padfeatures(reviewsInts, seqLength = seqLength)

assert len(features) == len(reviewsInts), "Fitur ini harus mempunyai banyak nya baris sesuai dengan review"
assert len(features[0]) == seqLength, "Setiap baris fitur harus terdapat nilai panjang sequence"

#cetak 10 nilai dari 30 batch
print(features[:30, :10])

[[    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [22382    42 46418    15   706 17139  3389    47    77    35]
 [ 4505   505    15     3  3342   162  8312  1652     6  4819]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [   54    10    14   116    60   798   552    71   364     5]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0]
 [    1   330   578    34     3   162   748  2731     9   325]
 [    9    11 10171  5305  1946   689   444    22   280   673]
 [    0     0     0     0     0     0     0     0     0